In [2]:
pip install keras-tcn

  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.10.0.dev20220617 requires flatbuffers>=2.0, but you have flatbuffers 1.12 which is incompatible.


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from sklearn.model_selection import train_test_split
from tcn import compiled_tcn

# 데이터 로드

In [2]:
curr_dir = os.getcwd()

forders_tr = os.listdir(curr_dir+"\\data")

data = pd.read_csv(os.path.join(curr_dir+"\\data\\")+forders_tr[0])
data = data.drop(data.columns[0], axis=1)
data

submission = pd.read_csv('2-1_검증데이터셋.csv')
submission1 = submission.copy()          

In [7]:
land = pd.read_csv('농업 지중온도.csv', encoding='cp949')
land1 = land[land['지점']==974]
land1 = land1.reset_index()
land1 = land1.drop('index', axis=1)
land1

,지점,지점명,일시,5CM 정시 지중온도(°C),10CM 정시 지중온도(°C),20CM 정시 지중온도(°C),30CM 정시 지중온도(°C),0.5M 정시 지중온도(°C),1.0M 정시 지중온도(°C),1.5M 정시 지중온도(°C),3.0M 정시 지중온도(°C),5.0M 정시 지중온도(°C)
0,974,대곡,2021-11-26 0:00,5.3,5.8,6.3,11.0,12.2,16.1,NaN,NaN,NaN
1,974,대곡,2021-11-26 1:00,4.5,5.1,5.7,10.9,12.2,16.1,NaN,NaN,NaN
2,974,대곡,2021-11-26 2:00,3.8,4.4,5.1,10.8,12.2,16.1,NaN,NaN,NaN
3,974,대곡,2021-11-26 3:00,3.3,3.9,4.6,10.6,12.2,16.0,NaN,NaN,NaN
4,974,대곡,2021-11-26 4:00,3.0,3.5,4.2,10.4,12.2,16.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2705,974,대곡,2022-03-20 19:00,10.6,10.8,10.2,10.0,9.7,11.2,NaN,NaN,NaN
2706,974,대곡,2022-03-20 20:00,9.6,9.9,9.5,10.1,9.8,11.2,NaN,NaN,NaN
2707,974,대곡,2022-03-20 21:00,8.8,9.1,8.8,10.1,9.8,11.1,NaN,NaN,NaN
2708,974,대곡,2022-03-20 22:00,8.1,8.3,8.2,10.2,9.9,11.2,NaN,NaN,NaN


In [16]:
land1.isna().sum()

지점                     0
지점명                    0
일시                     0
5CM 정시 지중온도(°C)        0
10CM 정시 지중온도(°C)       0
20CM 정시 지중온도(°C)       0
30CM 정시 지중온도(°C)       0
0.5M 정시 지중온도(°C)       0
1.0M 정시 지중온도(°C)       0
1.5M 정시 지중온도(°C)    2710
3.0M 정시 지중온도(°C)    2710
5.0M 정시 지중온도(°C)    2710
dtype: int64

In [19]:
land2 = land1[land1.columns[2:-3]]
land2

,일시,5CM 정시 지중온도(°C),10CM 정시 지중온도(°C),20CM 정시 지중온도(°C),30CM 정시 지중온도(°C),0.5M 정시 지중온도(°C),1.0M 정시 지중온도(°C)
0,2021-11-26 0:00,5.3,5.8,6.3,11.0,12.2,16.1
1,2021-11-26 1:00,4.5,5.1,5.7,10.9,12.2,16.1
2,2021-11-26 2:00,3.8,4.4,5.1,10.8,12.2,16.1
3,2021-11-26 3:00,3.3,3.9,4.6,10.6,12.2,16.0
4,2021-11-26 4:00,3.0,3.5,4.2,10.4,12.2,16.0
...,...,...,...,...,...,...,...
2705,2022-03-20 19:00,10.6,10.8,10.2,10.0,9.7,11.2
2706,2022-03-20 20:00,9.6,9.9,9.5,10.1,9.8,11.2
2707,2022-03-20 21:00,8.8,9.1,8.8,10.1,9.8,11.1
2708,2022-03-20 22:00,8.1,8.3,8.2,10.2,9.9,11.2


In [8]:
soil = pd.read_csv('토양수분.csv', encoding='cp949')
soil1 = soil[soil['지점']==974]
soil1 = soil1.reset_index()
soil1 = soil1.drop('index', axis=1)
soil1

,지점,지점명,일시,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%)
0,974,대곡,2021-11-26 00:00,11.2,7.9,4.0,5.0
1,974,대곡,2021-11-26 01:00,11.1,7.8,4.0,5.0
2,974,대곡,2021-11-26 02:00,11.0,7.8,4.0,5.0
3,974,대곡,2021-11-26 03:00,10.9,7.8,4.0,5.0
4,974,대곡,2021-11-26 04:00,10.8,7.8,3.9,5.0
...,...,...,...,...,...,...,...
2326,974,대곡,2022-03-20 19:00,16.4,17.8,12.4,12.8
2327,974,대곡,2022-03-20 20:00,16.3,17.6,12.4,12.8
2328,974,대곡,2022-03-20 21:00,16.3,17.6,12.4,12.8
2329,974,대곡,2022-03-20 22:00,16.1,17.5,12.4,12.8


In [20]:
soil2 = soil1[soil1.columns[2:]]
soil2

,일시,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%)
0,2021-11-26 00:00,11.2,7.9,4.0,5.0
1,2021-11-26 01:00,11.1,7.8,4.0,5.0
2,2021-11-26 02:00,11.0,7.8,4.0,5.0
3,2021-11-26 03:00,10.9,7.8,4.0,5.0
4,2021-11-26 04:00,10.8,7.8,3.9,5.0
...,...,...,...,...,...
2326,2022-03-20 19:00,16.4,17.8,12.4,12.8
2327,2022-03-20 20:00,16.3,17.6,12.4,12.8
2328,2022-03-20 21:00,16.3,17.6,12.4,12.8
2329,2022-03-20 22:00,16.1,17.5,12.4,12.8


In [23]:
land2.to_csv('land.csv', index=False)

In [24]:
soil2.to_csv('soil.csv', index=False)

In [111]:
soil3 = pd.read_csv('soil_1.csv', encoding='cp949')
soil3 = soil3.drop('일자', axis=1)


land3 = pd.read_csv('land_1.csv', encoding='cp949')
land3 = land3.drop('일자', axis=1)
land3

,5CM 정시 지중온도(°C),10CM 정시 지중온도(°C),20CM 정시 지중온도(°C),30CM 정시 지중온도(°C),0.5M 정시 지중온도(°C),1.0M 정시 지중온도(°C)
0,5.3,5.8,6.3,11.0,12.2,16.1
1,4.5,5.1,5.7,10.9,12.2,16.1
2,3.8,4.4,5.1,10.8,12.2,16.1
3,3.3,3.9,4.6,10.6,12.2,16.0
4,3.0,3.5,4.2,10.4,12.2,16.0
...,...,...,...,...,...,...
2755,10.6,10.8,10.2,10.0,9.7,11.2
2756,9.6,9.9,9.5,10.1,9.8,11.2
2757,8.8,9.1,8.8,10.1,9.8,11.1
2758,8.1,8.3,8.2,10.2,9.9,11.2


In [30]:
# 제공 데이터와 외부 데이터 통합 (시간 데이터 제외 모든 열)
col = ['smart_farm.insolation', 'smart_farm.out_tmperature',\
       'smart_farm.out_humidity', 'smart_farm.wind_speed',\
       'smart_farm.wind_direction', 'smart_farm.in_tmperature',\
       'smart_farm.in_humidity', 'smart_farm.shield_light_h',\
       'smart_farm.shield_tmperature_h', 'smart_farm.shield_energy_h',\
       'smart_farm.shield_energy_v', 'smart_farm.exhaust_fan',\
       'smart_farm.ceiling', 'smart_farm.floating_fan',\
       'smart_farm.fan_coil_b_site',\
       'smart_farm.ventilation_temperature_control',\
       'smart_farm.heating_temperature_set_up']          

data_ = pd.DataFrame()
for k in tqdm(range(len(col))):
    h = []
    for i in range(int(len(data)/60)):      # 분 -> 시간 단위 바꾸기 위해
        h.append(np.mean(data[col[k]][60*i:60*(i+1)]))
    data_[col[k]] = h
    
h1 = []
for i in range(int(len(data)/60)):
    h1.append(np.sum(data['smart_farm.heat_supply'][60*i:60*(i+1)]))
data_['smart_farm.heat_supply'] = h1

data_

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  2.97it/s]


,smart_farm.insolation,smart_farm.out_tmperature,smart_farm.out_humidity,smart_farm.wind_speed,smart_farm.wind_direction,smart_farm.in_tmperature,smart_farm.in_humidity,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,-3.221667,4.953333,70.383333,0.165000,229.550000,18.123333,61.928333,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.600000,28.000000,18.000000,46.8
1,-3.326667,3.186667,78.033333,0.006667,270.200000,18.088333,60.093333,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.650000,28.000000,18.000000,48.7
2,-3.553333,2.303333,81.900000,0.026667,260.500000,18.065000,59.030000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.683333,28.000000,18.000000,45.0
3,-3.753333,1.593333,83.416667,0.041667,248.266667,18.061667,58.215000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.750000,28.000000,18.000000,46.0
4,-3.736667,1.841667,80.516667,0.166667,266.850000,18.140351,57.308772,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.771930,28.000000,18.000000,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539,-2.696667,8.531667,55.366667,0.626667,157.850000,18.070833,81.993417,0.0,91.316667,0.0,0.0,0.0,0.0,1.000000,0.000000,27.866667,17.383333,0.0
2540,-2.505000,7.941667,56.900000,0.826667,253.183333,16.770833,81.953750,0.0,0.200000,0.0,0.0,0.0,0.0,0.983333,0.000000,28.000000,15.478333,0.0
2541,-2.288333,7.476667,57.616667,2.470000,311.216667,15.729167,80.955417,0.0,0.000000,0.0,0.0,0.0,0.0,0.966667,0.150000,28.000000,15.095000,0.0
2542,-2.210000,7.453333,54.883333,2.333333,239.250000,16.655000,78.901167,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.616667,28.000000,16.616667,0.0


In [102]:
# 21.12.22~24 / 22.03.1~3 / 22.03.14 / 22.03.17

a = list(pd.date_range('2021-11-26', periods=5*24+21*24, freq='1H'))
b = list(pd.date_range('2021-12-25', periods=7*24+31*24+28*24, freq='1H'))
c = list(pd.date_range('2022-03-04', periods=10*24, freq='1H'))
d = list(pd.date_range('2022-03-15', periods=24, freq='1H'))
e = list(pd.date_range('2022-03-18', periods=3*24, freq='1H'))

day = a+b+c+d+e

In [105]:
# data_['일자'] = day
data_.insert(0, '일자', day)

In [107]:
data_.to_csv('data.csv', index=False)

In [108]:
data3 = pd.read_csv('data_1.csv', encoding='cp949')
data3

,일자,smart_farm.insolation,smart_farm.out_tmperature,smart_farm.out_humidity,smart_farm.wind_speed,smart_farm.wind_direction,smart_farm.in_tmperature,smart_farm.in_humidity,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,2021-11-26 0:00,-3.221667,4.953333,70.383333,0.165000,229.550000,18.123333,61.928333,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.600000,28.000000,18.000000,46.8
1,2021-11-26 1:00,-3.326667,3.186667,78.033333,0.006667,270.200000,18.088333,60.093333,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.650000,28.000000,18.000000,48.7
2,2021-11-26 2:00,-3.553333,2.303333,81.900000,0.026667,260.500000,18.065000,59.030000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.683333,28.000000,18.000000,45.0
3,2021-11-26 3:00,-3.753333,1.593333,83.416667,0.041667,248.266667,18.061667,58.215000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.750000,28.000000,18.000000,46.0
4,2021-11-26 4:00,-3.736667,1.841667,80.516667,0.166667,266.850000,18.140351,57.308772,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.771930,28.000000,18.000000,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,2022-03-20 19:00,-2.696667,8.531667,55.366667,0.626667,157.850000,18.070833,81.993417,0.0,91.316667,0.0,0.0,0.0,0.0,1.000000,0.000000,27.866667,17.383333,0.0
2756,2022-03-20 20:00,-2.505000,7.941667,56.900000,0.826667,253.183333,16.770833,81.953750,0.0,0.200000,0.0,0.0,0.0,0.0,0.983333,0.000000,28.000000,15.478333,0.0
2757,2022-03-20 21:00,-2.288333,7.476667,57.616667,2.470000,311.216667,15.729167,80.955417,0.0,0.000000,0.0,0.0,0.0,0.0,0.966667,0.150000,28.000000,15.095000,0.0
2758,2022-03-20 22:00,-2.210000,7.453333,54.883333,2.333333,239.250000,16.655000,78.901167,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.616667,28.000000,16.616667,0.0


In [115]:
len(data3.columns) +len(land3.columns) +len(soil3.columns)

29

In [127]:
real = pd.concat([data3, land3, soil3], axis=1)
real1 = real.interpolate(method ='linear')  
# real1.isna().sum()
# real1[real1['smart_farm.heat_supply']==0]
# real1[-480:]['smart_farm.heat_supply']
real1.to_csv('지면_토양_대곡.csv', index=False)

# 전처리

In [121]:
soil = pd.read_csv('OBS_AAOS_TIM_20220802210224.csv', encoding='cp949')       # 외부 데이터 로드
stn = list(set(soil['지점'].values))

soil_df1 = []      # 일시 포함된 데이터 프레임
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    soil_df1.append(df)

    
soil_df = []        # 일시가 제외된 데이터 프레임 (추합을 위해)
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df = df.drop(['지점', '지점명', '일시'], axis=1)   
    soil_df.append(df) 
  
day = pd.date_range('2021-11-26', periods=95*24+20*24, freq='1H')    # 21.11.26~22.03.20
day
day1 = pd.DataFrame(day)
day1.columns = ['일자']
day1
day1.to_csv('day_real.csv', index=False)


for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df.to_csv(os.path.join(str(stn[i])+'.csv'), index=False)   # 관측소마다.csv 파일로 저장

In [251]:
# 관측소마다 일시(일자) 통합을 위해 'day_real'파일과 위에서 저장된 모든 관측소에 대해 엑셀 VLOOKUP 함수 이용하여 
# 중간에 비어 있는 일시를 채워주는 작업 후, 각 파일을 _1.csv 파일로 저장

soil_df2 = []
for i in range(len(stn)):
    df = pd.read_csv(os.path.join(str(stn[i])+'_1.csv'), encoding='cp949')
    df = df.drop('일자', axis=1)
    df = df.fillna(0)
    soil_df2.append(df)



soil_df3 = soil_df2[0].copy()
for i in range(len(soil_df[0].columns)):
    soil_df2[0][soil_df2[0].columns[i]] = 0

# 각 관측소마다 평균 내기 (추합)
for i in range(1, len(soil_df2)):
    soil_df3 = soil_df3 + soil_df2[i]
soil_df3 = soil_df3/len(soil_df2)
soil_df3['일자'] = pd.date_range('2021-11-26', periods=95*24+20*24, freq='1H')


# 제공 데이터와의 통합을 위해 제공 데이터에 없는 일시를 외부 데이터에서 제외시킴
d1 = soil_df3[soil_df3['일자'] == '2021-12-22 0:00'].index[0]
for i in range(d1, d1+24*3):
    soil_df3 = soil_df3.drop(i,axis=0)

d2 = soil_df3[soil_df3['일자'] == '2022-03-01 0:00'].index[0]
for i in range(d2, d2+24*3):
    soil_df3 = soil_df3.drop(i,axis=0)
    
d3 = soil_df3[soil_df3['일자'] == '2022-03-14 0:00'].index[0]
for i in range(d3, d3+24):
    soil_df3 = soil_df3.drop(i,axis=0)
    
d4 = soil_df3[soil_df3['일자'] == '2022-03-16 0:00'].index[0]
for i in range(d4, d4+24*2):
    soil_df3 = soil_df3.drop(i,axis=0)

soil_df3 = soil_df3.reset_index()
soil_df3 = soil_df3.drop('index', axis=1)

# 제공 데이터와 외부 데이터 통합 (시간 데이터 제외 모든 열)
col = ['smart_farm.insolation', 'smart_farm.out_tmperature',\
       'smart_farm.out_humidity', 'smart_farm.wind_speed',\
       'smart_farm.wind_direction', 'smart_farm.in_tmperature',\
       'smart_farm.in_humidity', 'smart_farm.shield_light_h',\
       'smart_farm.shield_tmperature_h', 'smart_farm.shield_energy_h',\
       'smart_farm.shield_energy_v', 'smart_farm.exhaust_fan',\
       'smart_farm.ceiling', 'smart_farm.floating_fan',\
       'smart_farm.fan_coil_b_site',\
       'smart_farm.ventilation_temperature_control',\
       'smart_farm.heating_temperature_set_up']          

data_ = pd.DataFrame()
for k in tqdm(range(len(col))):
    h = []
    for i in range(int(len(data)/60)):      # 분 -> 시간 단위 바꾸기 위해
        h.append(np.mean(data[col[k]][60*i:60*(i+1)]))
    data_[col[k]] = h
    
h1 = []
for i in range(int(len(data)/60)):
    h1.append(np.sum(data['smart_farm.heat_supply'][60*i:60*(i+1)]))
data_['smart_farm.heat_supply'] = h1



real_data = pd.concat([soil_df3, data_], axis=1)
real_data1 = pd.concat([real_data[real_data.columns[4]], real_data[real_data.columns[:4]], real_data[real_data.columns[5:]]], axis=1)
real_data1 = real_data1.reset_index()
real_data1 = real_data1.drop('index', axis=1)
real_data1 = real_data1.dropna(subset = ['일자'])
real_data1.to_csv('real_data_통합.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  2.86it/s]


In [3]:
# 위에서 저장한 'real_data_통합' 파일과 'day_real' 파일 이용하여
# 엑셀 VLOOKUP 함수 이용하여 21.11.26부터 22.03.20까지 비어 일자 채운 후, 다시 'real_data_결측' 파일로 저장

pre = pd.read_csv('real_data_결측.csv', encoding='cp949')
pre1 = pre.interpolate(method ='linear')      
pre1 = pre1.dropna(subset = ['일자'])
pre1.to_csv('real_data_최종.csv', index=False)  # 최종 전처리 파일(결측치 제거한)

real1 = pd.read_csv('real_data_최종.csv', encoding='utf-8')
real1

# 자른 이유? 20개를 추출하기 위해 일자 슬라이싱이 필요함.
training = real1[:-504]
testing = real1[-504:]
testing = testing.reset_index()
testing = testing.drop('index', axis=1)

# 모델 1

In [54]:
X = training.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)
y = training[['smart_farm.heat_supply']].values

training_data, test_data, y_tr, y_te = train_test_split(X, y, test_size=0.3, shuffle=False)  

validation = testing.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)


x_train = []
y_train = []

for i in range(24, training_data.shape[0]):
  x_train.append(training_data[i-24:i])
  y_train.append(training_data.iloc[i,-1])
    
    
x_test = []
y_test = []

for i in range(24, test_data.shape[0]):
  x_test.append(test_data[i-24:i])
  y_test.append(test_data.iloc[i,-1])
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)


x_validation = []
for i in range(24, validation.shape[0]):
  x_validation.append(validation[i-24:i])
x_validation = np.array(x_validation)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_validation.shape)


model2 = compiled_tcn(return_sequences=True,
                      num_feat=x_test.shape[2],
                      nb_filters=50,   
                      num_classes=0,
                      kernel_size=5,     
                      dilations=[2 ** i for i in range(9)],
                      nb_stacks=1,
                      max_len=x_test.shape[1],
                      use_skip_connections=True,
                      regression=True,
                      dropout_rate=0.3,    
                      output_len=1)   

model2.fit(x_train, y_train, batch_size=256, epochs=10)  

(1555, 24, 17)
(1555,)
(653, 24, 17)
(653,)
(480, 24, 17)
x.shape= (None, 24, 50)
model.x = (None, 24, 17)
model.y = (None, 24, 1)
Epoch 1/10


C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


7/7 [==============================] - 22s 2s/step - loss: 345346.5938
Epoch 2/10
7/7 [==============================] - 11s 2s/step - loss: 7990.3408
Epoch 3/10
7/7 [==============================] - 10s 1s/step - loss: 1185.1724
Epoch 4/10
7/7 [==============================] - 9s 1s/step - loss: 361.7487
Epoch 5/10
7/7 [==============================] - 9s 1s/step - loss: 118.6159
Epoch 6/10
7/7 [==============================] - 9s 1s/step - loss: 49.5551
Epoch 7/10
7/7 [==============================] - 9s 1s/step - loss: 25.1152
Epoch 8/10
7/7 [==============================] - 9s 1s/step - loss: 14.8116
Epoch 9/10
7/7 [==============================] - 10s 1s/step - loss: 10.7470
Epoch 10/10
7/7 [==============================] - 10s 1s/step - loss: 8.4017


In [56]:
p = model2.predict(x_validation)
p = p.flatten()

h1 = []
for i in range(int(len(p)/24)): 
    h1.append(np.sum(p[24*i:24*(i+1)]))    
    
    
h1.insert(0,0)     # 한 번만 클릭!
h11 = h1[4:14] + h1[18:21]
       
submission1['heat_supply_day1'] = h11
submission1 

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,393.216248,NaN
1,2022,3,5,396.483276,NaN
2,2022,3,6,396.800629,NaN
3,2022,3,7,396.489868,NaN
4,2022,3,8,391.743134,NaN
5,2022,3,9,384.124695,NaN
6,2022,3,10,384.696716,NaN
7,2022,3,11,385.992249,NaN
8,2022,3,12,383.541534,NaN
9,2022,3,13,374.856018,NaN


# 모델 2(절감 모델)

In [48]:
X = training.drop(['일자','smart_farm.heat_supply'], axis=1)
y = training[['smart_farm.heat_supply']].values

training_data, test_data, y_tr, y_te = train_test_split(X, y, test_size=0.3, shuffle=False)  

validation = testing.drop(['일자','smart_farm.heat_supply'], axis=1)


x_train = []
y_train = []

for i in range(24, training_data.shape[0]):
  x_train.append(training_data[i-24:i])
  y_train.append(training_data.iloc[i,-1])
    
x_test = []
y_test = []

for i in range(24, test_data.shape[0]): 
  x_test.append(test_data[i-24:i])
  y_test.append(test_data.iloc[i,-1])
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)



x_validation = []
for i in range(24, validation.shape[0]):
  x_validation.append(validation[i-24:i])
x_validation = np.array(x_validation)


print(x_train.shape) 
print(y_train.shape) 
print(x_test.shape)
print(y_test.shape)
print(x_validation.shape)

model = compiled_tcn(return_sequences=True,
                      num_feat=x_test.shape[2],
                      nb_filters=50,    
                      num_classes=0,
                      kernel_size=5,     
                      dilations=[2 ** i for i in range(9)],
                      nb_stacks=1,
                      max_len=x_test.shape[1],
                      use_skip_connections=True,
                      regression=True,
                      dropout_rate=0.3,     
                      output_len=1)  

model.fit(x_train, y_train, batch_size=256, epochs=10)   

(1555, 24, 21)
(1555,)
(653, 24, 21)
(653,)
(480, 24, 21)
x.shape= (None, 24, 50)
model.x = (None, 24, 21)
model.y = (None, 24, 1)
Epoch 1/10


C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


7/7 [==============================] - 21s 1s/step - loss: 386893.5938
Epoch 2/10
7/7 [==============================] - 11s 2s/step - loss: 11742.6504
Epoch 3/10
7/7 [==============================] - 10s 1s/step - loss: 2293.0818
Epoch 4/10
7/7 [==============================] - 11s 1s/step - loss: 784.1554
Epoch 5/10
7/7 [==============================] - 10s 1s/step - loss: 263.7638
Epoch 6/10
7/7 [==============================] - 11s 2s/step - loss: 102.9797
Epoch 7/10
7/7 [==============================] - 11s 2s/step - loss: 42.5303
Epoch 8/10
7/7 [==============================] - 16s 2s/step - loss: 21.2373
Epoch 9/10
7/7 [==============================] - 14s 2s/step - loss: 13.6550
Epoch 10/10
7/7 [==============================] - 13s 2s/step - loss: 10.5105


In [57]:
p2 = model.predict(x_validation)  
p2 = p2.flatten()

h2 = []
for i in range(int(len(p2)/24)): 
    h2.append(np.sum(p2[24*i:24*(i+1)]))
h2.insert(0,0)     # 한 번만 클릭!
h22 = h2[4:14] + h2[18:21]

submission1['heat_supply_day2'] = h22
submission1   

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,393.216248,363.161011
1,2022,3,5,396.483276,364.907593
2,2022,3,6,396.800629,367.740540
3,2022,3,7,396.489868,369.366730
4,2022,3,8,391.743134,368.829773
5,2022,3,9,384.124695,371.095886
6,2022,3,10,384.696716,372.659515
7,2022,3,11,385.992249,371.884674
8,2022,3,12,383.541534,366.714966
9,2022,3,13,374.856018,357.692993


In [58]:
submission1.to_csv('220224_최종코드틀_TCN_50_5_0.3_256_1등파일1.csv', index=False) 